# Environment Setup

In [3]:
import sys

if 'kaggle_web_client' in sys.modules:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    HUGGINGFACE_API_KEY = user_secrets.get_secret("HUGGINGFACE_API_KEY")
    WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()
    HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')
    WANDB_API_KEY = os.getenv("WANDB_API_KEY")

# Installing Required Libraries

In [4]:
!pip install -q accelerate evaluate rouge_score
!pip install -q datasets loralib einops
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

# Huggingface login

In [5]:
from huggingface_hub import notebook_login
from huggingface_hub import login
login(token=HUGGINGFACE_API_KEY)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# imports

In [6]:
from datasets import load_dataset, Dataset
import evaluate

import torch

import numpy as np
import pandas as pd
from tqdm.auto import tqdm


import bitsandbytes as bnb
from peft import PeftModel, PeftConfig


# from transformers import DataCollatorWithPadding, DataCollatorForLanguageModeling
# from transformers import T5ForConditionalGeneration, T5TokenizerFast
from transformers import AutoTokenizer, get_scheduler, BitsAndBytesConfig, GenerationConfig
from transformers import  AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
# from transformers import  AutoModelForCausalLM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading the data, tokenizer and Peft fine-tunned model

In [14]:
dataset = load_dataset("SKT27182/Preprocessed_OpenOrca", streaming=True, use_auth_token=True)
# samples = []
n_samples = 1000

data = {"id":[], "system_prompt":[], "question":[], "response": [], "length_before_preprocessing":[]}

for i, sample in enumerate(dataset["train"]):
    if i >= n_samples:
        break
    for key, value in sample.items():
        data[key].append(value)
        
open_orca = Dataset.from_dict(data)

In [15]:
# split the data for training and validation purpose
open_orca = open_orca.train_test_split(train_size=0.9, seed=42)

In [11]:
peft_model_id = "shirsh10mall/First_LLM_Project"
# peft_model_id = "SKT27182/Qlora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained( config.base_model_name_or_path, return_dict=True, load_in_8bit=True, 
                                                 device_map={"":0}, trust_remote_code=True, )

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
model

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-large and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear8bitLt(
                    in_features=1024, out_features=1024, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_e

# Zero shot Prediction


In [17]:
def analyse_zero_shot_model(data, indx, tokenizer, model, peft=False):
    prompt = data[indx]["system_prompt"]
    question = data[indx]["question"]
    response = data[indx]["response"]
    print("Prompt:")
    print(prompt)
    print()

    print('Question:')
    print(question)
    print()
    
    print("Response")
    print(response)
    print()
    
    tokenized_input = tokenizer(prompt, question, padding=True, truncation=True, return_tensors="pt")
    
    print("Tokenized Input: Prompt + Question")
    print(tokenized_input)
    print()
    
    print(tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0]))
    print()
    
    print("Zero-shot Prediction:")
    
    device = model.device
    
    if peft:
    
        predicted_response = model.generate(input_ids = tokenized_input["input_ids"].to(device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=50))
    else:
        predicted_response = model.generate(input_ids = tokenized_input["input_ids"].to(device))
    
    
    predicted_output = tokenizer.decode(predicted_response[0], skip_special_tokens=True)
    print(predicted_output)
    
    
index = 11
analyse_zero_shot_model(open_orca["train"], index, tokenizer, model, peft=True)

Prompt:
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.

Question:
Translate the following sentence to Russian:
Would you completely bet that it couldn't usurp Google, given time?

Response
To translate the given sentence into Russian, I'll analyze the sentence structure and words, identify the Russian counterparts, and then construct the Russian sentence while following the correct grammar and syntax rules. 

English sentence: Would you completely bet that it couldn't usurp Google, given time?

Translation process:
- "Would you" is a question construction that, in Russian, translates as "Вы бы" or "Ты бы" (the formal and informal ways to address "you," respectively; I'll go with the formal version)
- "completely bet" can be translated as "полностью поспорили"
- "that" as a conjunction introducing a clause can be translated as "что"
- "it" can be tran

In [20]:
index = 110
analyse_zero_shot_model(open_orca["train"], index, tokenizer, model, peft=True)

Prompt:
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.

Question:
Here is a review left by a customer on a product. Would you say he was satisfied or dissatisfied? Title: Michelle Bachman? Please Review: This book was made to make money off of others. WHat in the heck does michelle bachmann know?
A:

Response
Step 1: Analyze the title of the review
The title "Michelle Bachman?" suggests that the customer might be doubtful or confused about the subject matter, indicating negative emotions.

Step 2: Analyze the review content
In the review, the customer mentions that the book was made to make money off of others and questions what Michelle Bachman knows. This showcases dissatisfaction and disappointment with the book.

Step 3: Determine overall satisfaction
Based on the negative tone and questioning in both the title and the content of the review, I wo

In [24]:
index = 899
analyse_zero_shot_model(open_orca["train"], index, tokenizer, model, peft=True)

Prompt:
You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.

Question:

Q: I am verifying the answers generated by an automatic system to the following question: how deep can be drill for deep underwater Suggested answer: Oil spills result in oil sinking to the bottom of the ocean smothering plant life leaving fish without food and shelter to survive. Should I validate this answer?
A:

Response
No, the suggested answer does not appropriately respond to the question. The question is asking for the depth that can be drilled underwater, while the provided answer discusses the impact of oil spills in the ocean.

A more suitable answer would be: The depth that can be drilled underwater mainly depends on the technology being used. Currently, the most advanced deepwater drilling operations typically reach depths of 7,500 to 12,000 feet (2,300 to 3,660 meters) under the ocean surface. Ultra-deepwater drilling operations, on the othe

In [25]:
index = 655
analyse_zero_shot_model(open_orca["train"], index, tokenizer, model, peft=True)

Prompt:
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.

Question:
What most naturally follows?

A small group of people are seen playing lacrosse on a field running up and down the field and pushing one another. several shots of players hitting one another

Response
Step 1: Identify the context
Since the scenario describes a group of people playing lacrosse, it is clear that the context is a sports match or practice session. Lacrosse is a physically demanding and contact-intensive game.

Step 2: Understand the specific details
The description mentions that players are running up and down the field, pushing and hitting one another. This indicates that the players are actively engaged in the game and are using physical contact as allowed by the rules to gain an advantage.

Step 3: Determine a logical continuation
Based on the context and specific detai

---